## import dependencies

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load Cats vs Dogs dataset with train, validation, test splits

In [13]:
(train, validation, test), metadata = tfds.load(
                                        'cats_vs_dogs',
                                        split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
                                        with_info=True,
                                        as_supervised=True
                                      )
num_examples = metadata.splits['train'].num_examples
num_classes = metadata.features['label'].num_classes
print(num_examples)
print(num_classes)

23262
2


# Resize and normalize images

In [14]:
IMAGE_SIZE = (224, 224)

def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return image, label

# Prepare batches for training, validation, and testing

In [15]:
BATCH_SIZE = 32

## Training Batch
train_batches = train.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)

## Validation Batch
validation_batches = validation.map(format_image).batch(BATCH_SIZE).prefetch(1)

## Test batch
test_batches = test.map(format_image).batch(1)

# Load MobileNetV2 base model without top layers, pretrained on ImageNet

In [16]:
base_model = tf.keras.applications.MobileNetV2(
        input_shape=IMAGE_SIZE + (3,),
        include_top=False,
        weights='imagenet',
        pooling='avg'  # Global average pooling
    )

    # Freeze the base model
base_model.trainable = False

    # Build the model
model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

# Compile and train

In [17]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMAGE_SIZE + (3,),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Build the model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(), # Add a GlobalAveragePooling2D layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches, epochs=5, validation_data=validation_batches)

Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 53s 69ms/step - accuracy: 0.9515 - loss: 0.1187 - val_accuracy: 0.9811 - val_loss: 0.0559
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.9850 - loss: 0.0428 - val_accuracy: 0.9819 - val_loss: 0.0511
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 53ms/step - accuracy: 0.9868 - loss: 0.0400 - val_accuracy: 0.9828 - val_loss: 0.0517
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 39s 50ms/step - accuracy: 0.9879 - loss: 0.0366 - val_accuracy: 0.9819 - val_loss: 0.0524
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 39s 55ms/step - accuracy: 0.9906 - loss: 0.0281 - val_accuracy: 0.9837 - val_loss: 0.0554


# Save model in SavedModel format

In [28]:
PetVision_model = "/content/drive/MyDrive/PetVision-TFLite/pet_vision_model"
tf.saved_model.save(model, PetVision_model)

# Convert to TensorFlow Lite model

In [29]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_file = 'pet_vision_model.tflite'
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmp1f8n4ij9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_628')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  133141577331536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577330192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577331152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577330960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577332304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577331728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577330384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577332112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577330576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133141577333264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1331415773

# Load TFLite interpreter and allocate tensors

In [30]:
interpreter = tf.lite.Interpreter(model_path = tflite_model_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


# Run inference on first 100 test images using TFLite interpreter

In [31]:
test_labels, test_imgs = [], []
for img, label in test_batches.take(100):
  interpreter.set_tensor(input_index, img)
  interpreter.invoke()
  predictions.append(interpreter.get_tensor(output_index))
  test_labels.append(label.numpy()[0])
  test_imgs.append(img)

# Calculate number of correct predictions out of 100

In [32]:
score = 0
for item in range(0,99):
  prediction=np.argmax(predictions[item])
  label = test_labels[item]
  if prediction==label:
    score=score+1
print("Out of 100 predictions I got " + str(score) + " correct")

Out of 100 predictions I got 99 correct


# Define function to plot a single image with prediction info

In [40]:
def plot_image(i, predictions, true_labels, images):
    predictions_array = np.array(predictions[i][0])
    true_label = true_labels[i]
    img = images[i][0]

    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel(
        "Predicted: {} ({:2.0f}%)\nTrue: {}".format(
            predicted_label,
            100 * np.max(predictions_array),
            true_label
        ),
        color=color
    )


# Plot test images in a grid, 3 images per row, using existing plot_image function

In [41]:
num_images = 60
images_per_row = 3
rows = num_images // images_per_row + int(num_images % images_per_row > 0)

plt.figure(figsize=(5 * images_per_row, 4 * rows))

for i in range(num_images):
    plt.subplot(rows, images_per_row, i + 1)
    plot_image(i, predictions, test_labels, test_imgs)

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.